# LESSON 4: Working with DICOM Series (3D Volumes)
## Biomedical Image Processing - DICOM Module

In this lesson:
- Understanding DICOM series structure
- Loading multiple slices as a 3D volume
- Sorting slices correctly
- Visualizing 3D CT/MRI data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pydicom
from pydicom.dataset import Dataset, FileMetaDataset
from pydicom.uid import generate_uid
import os

print("Libraries imported successfully!")

## 1. DICOM Series Structure

A CT or MRI scan consists of multiple 2D slices that form a 3D volume:

```
Study (Patient Visit)
  └── Series 1 (e.g., CT Axial)
        ├── Slice 1 (image_001.dcm)
        ├── Slice 2 (image_002.dcm)
        ├── Slice 3 (image_003.dcm)
        └── ... (hundreds of slices)
  └── Series 2 (e.g., CT Coronal)
        └── ...
```

### Key Concepts:
- **SeriesInstanceUID**: Unique identifier for a series
- **InstanceNumber**: Slice number within series
- **ImagePositionPatient**: 3D position of the slice
- **SliceLocation**: Z-position of the slice

## 2. Creating a Sample DICOM Series

Let's create a synthetic 3D CT volume with multiple slices.

In [ ]:
def create_3d_phantom(size=64, num_slices=20):
    """
    Create a 3D phantom with anatomical-like structures.
    Returns pixel values in stored format (add 1024 for HU).
    """
    volume = np.zeros((num_slices, size, size), dtype=np.int16)
    
    # Background: Air (-1000 HU -> stored 24)
    volume[:] = 24
    
    z, y, x = np.ogrid[:num_slices, :size, :size]
    center = size // 2
    z_center = num_slices // 2
    
    # Body ellipsoid (soft tissue ~40 HU -> stored 1064)
    body = ((x - center)**2 / (center*0.8)**2 + 
            (y - center)**2 / (center*0.9)**2 + 
            (z - z_center)**2 / (z_center*0.9)**2) < 1
    volume[body] = 1064 + np.random.randint(-20, 20, np.sum(body))
    
    # Spine (bone ~500 HU -> stored 1524)
    spine_radius = size // 10
    spine = ((x - center)**2 + (y - int(center*1.4))**2) < spine_radius**2
    spine = spine & body
    volume[spine] = 1524 + np.random.randint(-50, 50, np.sum(spine))
    
    # Lungs (air ~-500 HU -> stored 524) - only in middle slices
    lung_slices = (z > num_slices*0.2) & (z < num_slices*0.8)
    lung_left = ((x - int(center*0.6))**2 + (y - center)**2) < (size//6)**2
    lung_right = ((x - int(center*1.4))**2 + (y - center)**2) < (size//6)**2
    lungs = lung_slices & (lung_left | lung_right) & body
    volume[lungs] = 524 + np.random.randint(-50, 50, np.sum(lungs))
    
    # Heart (blood ~50 HU -> stored 1074) - central region
    heart_slices = (z > num_slices*0.3) & (z < num_slices*0.7)
    heart = ((x - center)**2 + (y - int(center*0.9))**2) < (size//8)**2
    heart = heart_slices & heart & body
    volume[heart] = 1074 + np.random.randint(-10, 10, np.sum(heart))
    
    return volume

# Create 3D phantom
phantom_3d = create_3d_phantom(size=64, num_slices=20)
print(f"3D Phantom shape: {phantom_3d.shape}")
print(f"Value range: [{phantom_3d.min()}, {phantom_3d.max()}]")

In [ ]:
def create_dicom_series(volume, output_dir, series_description="CT Series"):
    """
    Create a DICOM series from a 3D volume.
    
    Parameters:
    -----------
    volume : ndarray
        3D array (slices, height, width)
    output_dir : str
        Directory to save DICOM files
    series_description : str
        Description of the series
    """
    os.makedirs(output_dir, exist_ok=True)
    
    num_slices, rows, cols = volume.shape
    
    # Generate UIDs (same for all slices in series)
    study_uid = generate_uid()
    series_uid = generate_uid()
    frame_of_ref_uid = generate_uid()
    
    slice_thickness = 2.5  # mm
    pixel_spacing = [1.0, 1.0]  # mm
    
    for i in range(num_slices):
        # Create file meta
        file_meta = FileMetaDataset()
        file_meta.MediaStorageSOPClassUID = pydicom.uid.CTImageStorage
        file_meta.MediaStorageSOPInstanceUID = generate_uid()
        file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
        file_meta.ImplementationClassUID = generate_uid()
        
        # Create dataset
        ds = Dataset()
        ds.file_meta = file_meta
        ds.is_little_endian = True
        ds.is_implicit_VR = False
        ds.preamble = b'\x00' * 128
        
        # Patient
        ds.PatientName = "Phantom^3D"
        ds.PatientID = "PHANTOM001"
        ds.PatientBirthDate = "19900101"
        ds.PatientSex = "O"
        
        # Study
        ds.StudyDate = "20240120"
        ds.StudyTime = "120000"
        ds.StudyDescription = "3D Phantom Study"
        ds.StudyInstanceUID = study_uid
        ds.StudyID = "1"
        
        # Series
        ds.Modality = "CT"
        ds.SeriesDescription = series_description
        ds.SeriesInstanceUID = series_uid
        ds.SeriesNumber = 1
        
        # Frame of Reference
        ds.FrameOfReferenceUID = frame_of_ref_uid
        ds.PositionReferenceIndicator = ""
        
        # Image
        ds.SOPClassUID = pydicom.uid.CTImageStorage
        ds.SOPInstanceUID = file_meta.MediaStorageSOPInstanceUID
        ds.InstanceNumber = i + 1
        ds.ImageType = ["ORIGINAL", "PRIMARY", "AXIAL"]
        
        # Position and Orientation
        z_position = i * slice_thickness
        ds.ImagePositionPatient = [0, 0, z_position]
        ds.ImageOrientationPatient = [1, 0, 0, 0, 1, 0]
        ds.SliceLocation = z_position
        ds.SliceThickness = slice_thickness
        ds.PixelSpacing = pixel_spacing
        
        # Pixel Data
        ds.Rows = rows
        ds.Columns = cols
        ds.BitsAllocated = 16
        ds.BitsStored = 16
        ds.HighBit = 15
        ds.PixelRepresentation = 1
        ds.SamplesPerPixel = 1
        ds.PhotometricInterpretation = "MONOCHROME2"
        
        # Rescale
        ds.RescaleIntercept = -1024
        ds.RescaleSlope = 1
        
        # Window
        ds.WindowCenter = 40
        ds.WindowWidth = 400
        
        # Set pixel data
        ds.PixelData = volume[i].tobytes()
        
        # Save
        filename = os.path.join(output_dir, f"slice_{i+1:03d}.dcm")
        pydicom.dcmwrite(filename, ds, write_like_original=False)
    
    print(f"Created {num_slices} DICOM files in '{output_dir}'")
    return output_dir

# Create DICOM series
series_dir = create_dicom_series(phantom_3d, "sample_series", "Axial CT")

## 3. Loading a DICOM Series

In [ ]:
def load_dicom_series(directory):
    """
    Load all DICOM files from a directory and return as 3D volume.
    
    Parameters:
    -----------
    directory : str
        Path to directory containing DICOM files
        
    Returns:
    --------
    tuple: (volume, slices) - 3D numpy array and list of datasets
    """
    # Find all DICOM files
    dicom_files = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        try:
            ds = pydicom.dcmread(filepath)
            dicom_files.append(ds)
        except:
            continue
    
    if not dicom_files:
        raise ValueError(f"No DICOM files found in {directory}")
    
    print(f"Found {len(dicom_files)} DICOM files")
    
    # Sort by SliceLocation or InstanceNumber
    try:
        dicom_files.sort(key=lambda x: float(x.SliceLocation))
        print("Sorted by SliceLocation")
    except:
        dicom_files.sort(key=lambda x: int(x.InstanceNumber))
        print("Sorted by InstanceNumber")
    
    # Get dimensions
    first_slice = dicom_files[0]
    rows = first_slice.Rows
    cols = first_slice.Columns
    num_slices = len(dicom_files)
    
    print(f"Volume dimensions: {num_slices} x {rows} x {cols}")
    
    # Create 3D array
    volume = np.zeros((num_slices, rows, cols), dtype=np.float32)
    
    for i, ds in enumerate(dicom_files):
        # Get pixel array and apply rescaling
        pixel_array = ds.pixel_array.astype(float)
        slope = getattr(ds, 'RescaleSlope', 1)
        intercept = getattr(ds, 'RescaleIntercept', 0)
        volume[i] = pixel_array * slope + intercept
    
    return volume, dicom_files

# Load the series
volume, slices = load_dicom_series("sample_series")
print(f"\nLoaded volume shape: {volume.shape}")
print(f"HU range: [{volume.min():.0f}, {volume.max():.0f}]")

## 4. Extracting Series Metadata

In [ ]:
def get_series_info(slices):
    """Extract important information from a DICOM series."""
    first = slices[0]
    
    # Get pixel spacing and slice thickness
    pixel_spacing = getattr(first, 'PixelSpacing', [1, 1])
    slice_thickness = getattr(first, 'SliceThickness', 1)
    
    # Calculate slice spacing from positions
    if len(slices) > 1:
        pos1 = getattr(slices[0], 'ImagePositionPatient', [0, 0, 0])
        pos2 = getattr(slices[1], 'ImagePositionPatient', [0, 0, 1])
        slice_spacing = abs(pos2[2] - pos1[2])
    else:
        slice_spacing = slice_thickness
    
    info = {
        'patient_name': str(getattr(first, 'PatientName', 'Unknown')),
        'patient_id': str(getattr(first, 'PatientID', 'Unknown')),
        'modality': str(getattr(first, 'Modality', 'Unknown')),
        'series_description': str(getattr(first, 'SeriesDescription', '')),
        'num_slices': len(slices),
        'rows': first.Rows,
        'columns': first.Columns,
        'pixel_spacing': [float(pixel_spacing[0]), float(pixel_spacing[1])],
        'slice_thickness': float(slice_thickness),
        'slice_spacing': float(slice_spacing),
        'total_height_mm': len(slices) * slice_spacing,
    }
    
    # Calculate voxel size
    info['voxel_size'] = [
        info['pixel_spacing'][0],
        info['pixel_spacing'][1],
        info['slice_spacing']
    ]
    
    return info

series_info = get_series_info(slices)

print("SERIES INFORMATION")
print("=" * 40)
for key, value in series_info.items():
    print(f"{key:20s}: {value}")

## 5. Visualizing 3D Volume - Axial, Sagittal, Coronal Views

In [ ]:
def show_orthogonal_views(volume, slice_indices=None, window_center=40, window_width=400):
    """
    Show axial, sagittal, and coronal views of a 3D volume.
    
    Parameters:
    -----------
    volume : ndarray
        3D array (slices, rows, cols)
    slice_indices : tuple
        (axial, sagittal, coronal) indices. If None, uses center.
    window_center, window_width : float
        Window/level settings
    """
    nz, ny, nx = volume.shape
    
    if slice_indices is None:
        slice_indices = (nz // 2, ny // 2, nx // 2)
    
    ax_idx, sag_idx, cor_idx = slice_indices
    
    # Window settings
    vmin = window_center - window_width / 2
    vmax = window_center + window_width / 2
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Axial view (top-down)
    axes[0].imshow(volume[ax_idx, :, :], cmap='gray', vmin=vmin, vmax=vmax)
    axes[0].set_title(f'Axial (Slice {ax_idx + 1}/{nz})')
    axes[0].set_xlabel('Left - Right')
    axes[0].set_ylabel('Anterior - Posterior')
    axes[0].axhline(cor_idx, color='r', linestyle='--', alpha=0.5)
    axes[0].axvline(sag_idx, color='b', linestyle='--', alpha=0.5)
    
    # Sagittal view (side view)
    axes[1].imshow(volume[:, :, sag_idx], cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
    axes[1].set_title(f'Sagittal (X = {sag_idx})')
    axes[1].set_xlabel('Anterior - Posterior')
    axes[1].set_ylabel('Superior - Inferior')
    axes[1].axhline(ax_idx, color='g', linestyle='--', alpha=0.5)
    axes[1].axvline(cor_idx, color='r', linestyle='--', alpha=0.5)
    
    # Coronal view (front view)
    axes[2].imshow(volume[:, cor_idx, :], cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
    axes[2].set_title(f'Coronal (Y = {cor_idx})')
    axes[2].set_xlabel('Left - Right')
    axes[2].set_ylabel('Superior - Inferior')
    axes[2].axhline(ax_idx, color='g', linestyle='--', alpha=0.5)
    axes[2].axvline(sag_idx, color='b', linestyle='--', alpha=0.5)
    
    plt.suptitle('Orthogonal Views (Axial, Sagittal, Coronal)', fontsize=14)
    plt.tight_layout()
    plt.show()

# Show orthogonal views
show_orthogonal_views(volume)

In [ ]:
# Show with different windows
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

mid_slice = volume.shape[0] // 2

# Lung window
axes[0].imshow(volume[mid_slice], cmap='gray', vmin=-1200, vmax=200)
axes[0].set_title('Lung Window\nW:1400 L:-500')
axes[0].axis('off')

# Soft tissue window
axes[1].imshow(volume[mid_slice], cmap='gray', vmin=-160, vmax=240)
axes[1].set_title('Soft Tissue Window\nW:400 L:40')
axes[1].axis('off')

# Bone window
axes[2].imshow(volume[mid_slice], cmap='gray', vmin=-500, vmax=1300)
axes[2].set_title('Bone Window\nW:1800 L:400')
axes[2].axis('off')

plt.suptitle(f'Slice {mid_slice + 1} with Different Windows', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Scrolling Through Slices

In [ ]:
def show_slice_montage(volume, num_cols=5, window_center=40, window_width=400):
    """
    Show all slices as a montage.
    """
    num_slices = volume.shape[0]
    num_rows = (num_slices + num_cols - 1) // num_cols
    
    vmin = window_center - window_width / 2
    vmax = window_center + window_width / 2
    
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(3*num_cols, 3*num_rows))
    axes = axes.flatten()
    
    for i in range(len(axes)):
        if i < num_slices:
            axes[i].imshow(volume[i], cmap='gray', vmin=vmin, vmax=vmax)
            axes[i].set_title(f'Slice {i+1}', fontsize=10)
        axes[i].axis('off')
    
    plt.suptitle('All Slices Montage', fontsize=14)
    plt.tight_layout()
    plt.show()

show_slice_montage(volume, num_cols=5)

## 7. Maximum Intensity Projection (MIP)

In [ ]:
def maximum_intensity_projection(volume):
    """
    Create Maximum Intensity Projections (MIP) along all three axes.
    MIP shows the maximum value along each ray through the volume.
    Useful for visualizing high-intensity structures (vessels, bones).
    """
    # MIP along each axis
    mip_axial = np.max(volume, axis=0)      # From top
    mip_sagittal = np.max(volume, axis=2)   # From side
    mip_coronal = np.max(volume, axis=1)    # From front
    
    return mip_axial, mip_sagittal, mip_coronal

mip_ax, mip_sag, mip_cor = maximum_intensity_projection(volume)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Use bone window for MIP (to see high intensity structures)
vmin, vmax = -200, 800

axes[0].imshow(mip_ax, cmap='gray', vmin=vmin, vmax=vmax)
axes[0].set_title('MIP - Axial (Top View)')
axes[0].axis('off')

axes[1].imshow(mip_sag, cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
axes[1].set_title('MIP - Sagittal (Side View)')
axes[1].axis('off')

axes[2].imshow(mip_cor, cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
axes[2].set_title('MIP - Coronal (Front View)')
axes[2].axis('off')

plt.suptitle('Maximum Intensity Projections', fontsize=14)
plt.tight_layout()
plt.show()

## 8. Volume Statistics and Histogram

In [ ]:
def analyze_volume(volume):
    """Calculate statistics for a 3D volume."""
    stats = {
        'shape': volume.shape,
        'total_voxels': volume.size,
        'min_hu': volume.min(),
        'max_hu': volume.max(),
        'mean_hu': volume.mean(),
        'std_hu': volume.std(),
        'median_hu': np.median(volume),
    }
    
    # Count tissue types based on HU ranges
    stats['air_voxels'] = np.sum(volume < -500)
    stats['lung_voxels'] = np.sum((volume >= -500) & (volume < -100))
    stats['fat_voxels'] = np.sum((volume >= -100) & (volume < -10))
    stats['soft_tissue_voxels'] = np.sum((volume >= -10) & (volume < 100))
    stats['bone_voxels'] = np.sum(volume >= 100)
    
    return stats

stats = analyze_volume(volume)

print("VOLUME STATISTICS")
print("=" * 40)
print(f"Shape: {stats['shape']}")
print(f"Total voxels: {stats['total_voxels']:,}")
print(f"\nHounsfield Unit Statistics:")
print(f"  Min: {stats['min_hu']:.1f} HU")
print(f"  Max: {stats['max_hu']:.1f} HU")
print(f"  Mean: {stats['mean_hu']:.1f} HU")
print(f"  Std: {stats['std_hu']:.1f} HU")
print(f"  Median: {stats['median_hu']:.1f} HU")
print(f"\nTissue Distribution:")
print(f"  Air (<-500 HU): {stats['air_voxels']:,} voxels")
print(f"  Lung (-500 to -100 HU): {stats['lung_voxels']:,} voxels")
print(f"  Fat (-100 to -10 HU): {stats['fat_voxels']:,} voxels")
print(f"  Soft Tissue (-10 to 100 HU): {stats['soft_tissue_voxels']:,} voxels")
print(f"  Bone (>100 HU): {stats['bone_voxels']:,} voxels")

In [ ]:
# Plot histogram
fig, ax = plt.subplots(figsize=(12, 5))

hist, bins = np.histogram(volume.flatten(), bins=100, range=(-1000, 1000))
ax.bar(bins[:-1], hist, width=np.diff(bins), edgecolor='none', alpha=0.7)

# Add tissue region markers
ax.axvspan(-1000, -500, alpha=0.2, color='blue', label='Air')
ax.axvspan(-500, -100, alpha=0.2, color='cyan', label='Lung')
ax.axvspan(-100, -10, alpha=0.2, color='yellow', label='Fat')
ax.axvspan(-10, 100, alpha=0.2, color='red', label='Soft Tissue')
ax.axvspan(100, 1000, alpha=0.2, color='white', label='Bone')

ax.set_xlabel('Hounsfield Units')
ax.set_ylabel('Voxel Count')
ax.set_title('Volume Histogram with Tissue Regions')
ax.legend(loc='upper right')
ax.set_xlim(-1100, 1100)

plt.tight_layout()
plt.show()

In [ ]:
# Clean up - remove the sample series
import shutil
if os.path.exists('sample_series'):
    shutil.rmtree('sample_series')
    print("Sample series cleaned up.")

## Summary

What we learned:
1. DICOM series consist of multiple 2D slices forming a 3D volume
2. **Sort slices** by SliceLocation or InstanceNumber
3. Extract **voxel spacing** from PixelSpacing and SliceThickness
4. View **orthogonal planes**: Axial, Sagittal, Coronal
5. **MIP** (Maximum Intensity Projection) for vessel/bone visualization
6. Analyze **tissue distribution** using HU thresholds